In [1]:
import pandas as pd
import numpy as np
import glob
import os
import pickle

In [2]:
with open('D:\Weather Project\data\MET Data\complete_data.pkl', 'rb') as file:
    df = pickle.load(file)

df = df[['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)','REGIAO', 'UF', 'ESTACAO',
       'CODIGO (WMO)']]

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_4744\1666682795.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\complete_data.pkl', 'rb') as file:


In [3]:
# Drop cities with only missing values
nan_percentages_by_city = df.groupby('CODIGO (WMO)')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].apply(lambda x: (x.isna().sum() / len(x)))
missing_idxs = nan_percentages_by_city[nan_percentages_by_city> 0.95].index
df = df[~df['CODIGO (WMO)'].isin(missing_idxs)]

In [4]:
df['lag_precip'] = df.groupby('CODIGO (WMO)')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].shift(1)
df['rain_start_count'] = 0
df.loc[(df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] > 0) & (df['lag_precip']==0), 'rain_start_count'] = 1

In [5]:
with open('D:\Weather Project\data\MET Data\precip_data.pkl', 'wb') as file:
    pickle.dump(df, file)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_4744\1792589524.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\precip_data.pkl', 'wb') as file:


In [6]:
with open('D:\Weather Project\data\MET Data\glossary.pkl', 'rb') as file:
    glossary = pickle.load(file)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_4744\173438111.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\glossary.pkl', 'rb') as file:


In [7]:
city_week_group = df.groupby(['CODIGO (WMO)', pd.Grouper(freq='W', key='Data')])[['rain_start_count']].sum()
city_week_group.reset_index(inplace=True)

In [8]:
city_group = city_week_group.groupby('CODIGO (WMO)').mean(numeric_only=True)
city_group.rename(columns={
    'rain_start_count':'mean weekly rain starts'
}, inplace =True)

city_group = city_group.merge(glossary, left_index=True, right_index=True)

In [9]:
# Create a new column with the sorted order of 'Column1' in descending order
city_group['Precip Raning'] = (-city_group['mean weekly rain starts']).argsort().argsort()

In [10]:
city_group

,mean weekly rain starts,REGIAO,UF,ESTACAO,LATITUDE,LONGITUDE,ALTITUDE,Precip Raning
CODIGO (WMO),,,,,,,,
A001,4.343949,CO,DF,BRASILIA,"-15,78944444","-47,92583332","1160,96",162
A002,4.694268,CO,GO,GOIANIA,"-16,642841","-49,220222","727,3",121
A003,4.439490,CO,GO,MORRINHOS,"-17,745066","-49,101698","751,09",149
A005,3.267516,CO,GO,PORANGATU,"-13,30944444","-49,11749999",365,268
A009,4.152866,N,TO,PALMAS,"-10,19083333","-48,30194444","291,87",177
...,...,...,...,...,...,...,...,...
S712,2.830189,CO,MS,NOVA ALVORADA DO SUL,"-21,450972","-54,341972",5,313
S713,0.528302,CO,MS,NOVA ANDRADINA,"-22,0786111","-53,46583333",372,541
S714,1.886792,CO,MS,PEDRO GOMES,"-18,07277777","-54,54888888",251,425


In [11]:
with open('D:\Weather Project\data\MET Data\city_group.pkl', 'wb') as file:
    pickle.dump(city_group, file)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\marti\AppData\Local\Temp\ipykernel_4744\4290103485.py:1: SyntaxWarning: invalid escape sequence '\W'
  with open('D:\Weather Project\data\MET Data\city_group.pkl', 'wb') as file:
